<a href="https://colab.research.google.com/github/Rushi1109/Dogs-Vs-Cats-CNN/blob/main/dogs_vs_cat_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt-get update
!sudo apt-get install -y libopencv-dev

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,149 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,830 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,532 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-b

In [2]:
!wget https://download.pytorch.org/libtorch/cu126/libtorch-shared-with-deps-2.9.1%2Bcu126.zip

--2025-11-20 06:25:22--  https://download.pytorch.org/libtorch/cu126/libtorch-shared-with-deps-2.9.1%2Bcu126.zip
Resolving download.pytorch.org (download.pytorch.org)... 3.169.252.101, 3.169.252.20, 3.169.252.18, ...
Connecting to download.pytorch.org (download.pytorch.org)|3.169.252.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3221121553 (3.0G) [binary/octet-stream]
Saving to: ‘libtorch-shared-with-deps-2.9.1+cu126.zip’

libtorch-shared-wit 100%[===================>]   3.00G   146MB/s    in 19s     

2025-11-20 06:25:41 (165 MB/s) - ‘libtorch-shared-with-deps-2.9.1+cu126.zip’ saved [3221121553/3221121553]



In [3]:
!unzip libtorch-shared-with-deps-2.9.1+cu126.zip

Streaming output truncated to the last 5000 lines.
  inflating: libtorch/include/ATen/ops/max_pool1d_with_indices_native.h  
  inflating: libtorch/include/ATen/ops/max_pool1d_with_indices_ops.h  
  inflating: libtorch/include/ATen/ops/max_pool2d.h  
  inflating: libtorch/include/ATen/ops/max_pool2d_backward.h  
  inflating: libtorch/include/ATen/ops/max_pool2d_backward_compositeexplicitautograd_dispatch.h  
  inflating: libtorch/include/ATen/ops/max_pool2d_backward_native.h  
  inflating: libtorch/include/ATen/ops/max_pool2d_backward_ops.h  
  inflating: libtorch/include/ATen/ops/max_pool2d_compositeimplicitautograd_dispatch.h  
  inflating: libtorch/include/ATen/ops/max_pool2d_native.h  
  inflating: libtorch/include/ATen/ops/max_pool2d_ops.h  
  inflating: libtorch/include/ATen/ops/max_pool2d_with_indices.h  
  inflating: libtorch/include/ATen/ops/max_pool2d_with_indices_backward.h  
  inflating: libtorch/include/ATen/ops/max_pool2d_with_indices_backward_compositeexplicitautogradnonf

In [4]:
!mkdir -p ~/.kaggle/

In [24]:
%%writefile ~/.kaggle/kaggle.json
{"username":"your_kaggle_username","key":"your_kaggle_apikey"}

Overwriting /root/.kaggle/kaggle.json


In [6]:
!curl -L -o /content/dogsvscats.zip https://www.kaggle.com/api/v1/datasets/download/princelv84/dogsvscats

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  544M  100  544M    0     0   116M      0  0:00:04  0:00:04 --:--:--  128M


In [7]:
!mkdir -p /content/datasets/dogs-vs-cats/

In [8]:
!unzip dogsvscats.zip -d /content/datasets/dogs-vs-cats/

Streaming output truncated to the last 5000 lines.
  inflating: /content/datasets/dogs-vs-cats/train/dogs/dog.4419.jpg  
  inflating: /content/datasets/dogs-vs-cats/train/dogs/dog.442.jpg  
  inflating: /content/datasets/dogs-vs-cats/train/dogs/dog.4420.jpg  
  inflating: /content/datasets/dogs-vs-cats/train/dogs/dog.4421.jpg  
  inflating: /content/datasets/dogs-vs-cats/train/dogs/dog.4422.jpg  
  inflating: /content/datasets/dogs-vs-cats/train/dogs/dog.4424.jpg  
  inflating: /content/datasets/dogs-vs-cats/train/dogs/dog.4425.jpg  
  inflating: /content/datasets/dogs-vs-cats/train/dogs/dog.4426.jpg  
  inflating: /content/datasets/dogs-vs-cats/train/dogs/dog.4427.jpg  
  inflating: /content/datasets/dogs-vs-cats/train/dogs/dog.4431.jpg  
  inflating: /content/datasets/dogs-vs-cats/train/dogs/dog.4433.jpg  
  inflating: /content/datasets/dogs-vs-cats/train/dogs/dog.4436.jpg  
  inflating: /content/datasets/dogs-vs-cats/train/dogs/dog.4438.jpg  
  inflating: /content/datasets/dogs-vs-c

In [22]:
%%writefile Cats-Vs-Dogs-Classification.cpp
// Dataset URL: https://www.kaggle.com/datasets/princelv84/dogsvscats?resource=download
// We'll create a CNN to classify images of cats and dogs.

#include <torch/torch.h>
#include <torch/serialize/input-archive.h>
#include <torch/serialize/output-archive.h>
#include <filesystem>
#include <iostream>
#include <fstream>
#include <opencv2/opencv.hpp>

namespace fs = std::filesystem;
torch::Device device(torch::kCPU);
static inline std::map<std::string, uint8_t> CLASS_TO_LABEL = {
    {"cat", 0},
    {"dog", 1}
};

struct LoaderConfig {
    int batch_size;
    int workers;
};

LoaderConfig get_loader_config()
{
    bool cuda = torch::cuda::is_available();
    bool is_linux =
#if defined(__linux__)
        true;
#else
        false;
#endif

    LoaderConfig cfg;

    if (is_linux && cuda) {
        // Google Colab GPU detected
        cfg.batch_size = 64;
        cfg.workers    = 4;
    }
    else if (is_linux && !cuda) {
        // Colab CPU mode
        cfg.batch_size = 32;
        cfg.workers    = 2;
    }
    else {
        // Local machine: Windows or macOS
        cfg.batch_size = 16;
        cfg.workers    = 0;  // safer because OpenCV & LibTorch sometimes break with threads on Windows
    }

    return cfg;
}

void load_dataset_folder(const fs::path &rootDir, std::vector<fs::path> &paths, std::vector<uint8_t> &labels, std::map<std::string, uint8_t> &classToLabel)
{
    if (!fs::exists(rootDir))
        return;
    for (auto &classDir : fs::directory_iterator(rootDir))
    {
        if (!classDir.is_directory())
            continue;

        std::string className = classDir.path().filename().string();

        // Normalize class name
        std::string name = classDir.path().filename().string();
        std::transform(name.begin(), name.end(), name.begin(), ::tolower);

        uint8_t label;
        if (name.find("cat") != std::string::npos)
            label = CLASS_TO_LABEL["cat"];
        else if (name.find("dog") != std::string::npos)
            label = CLASS_TO_LABEL["dog"];
        else
            continue; // skip unknown folder

        for (auto &imgFile : fs::directory_iterator(classDir.path()))
        {
            if (imgFile.is_regular_file())
            {
                paths.push_back(imgFile.path());
                labels.push_back(label);
            }
        }
    }
}

class ImageDataset : public torch::data::datasets::Dataset<ImageDataset>
{
public:
    ImageDataset(std::vector<fs::path> imagePaths, std::vector<uint8_t> labels, int targetHeight = 256, int targetWidth = 256)
        : m_imagePaths(std::move(imagePaths)), m_labels(std::move(labels)), m_targetHeight(targetHeight), m_targetWidth(targetWidth) {}

    torch::data::Example<> get(size_t index) override
    {
        const fs::path &imgPath = m_imagePaths.at(index);

        cv::Mat img = cv::imread(imgPath.string(), cv::IMREAD_COLOR);
        if (img.empty())
        {
            throw std::runtime_error("Could not open or find the image: " + imgPath.string());
        }

        cv::cvtColor(img, img, cv::COLOR_BGR2RGB);
        cv::resize(img, img, cv::Size(m_targetWidth, m_targetHeight));

        torch::Tensor imgTensor = torch::from_blob(img.data, {m_targetHeight, m_targetWidth, 3}, torch::kByte).to(torch::kFloat32) / 255.0;
        imgTensor = imgTensor.permute({2, 0, 1}); // HxWxC to CxHxW
        int label = m_labels[index];

        return {imgTensor, torch::tensor(label)};
    }

    torch::optional<size_t> size() const override
    {
        return m_imagePaths.size();
    }

private:
    std::vector<fs::path> m_imagePaths;
    std::vector<uint8_t> m_labels;
    int m_targetHeight, m_targetWidth;
};

class CNNImpl : public torch::nn::Module
{
public:
    CNNImpl()
    {
        // ----- Feature Extractor -----
        conv1 = register_module("conv1", torch::nn::Conv2d(3, 32, 3));
        bn1   = register_module("bn1", torch::nn::BatchNorm2d(32));
        pool1 = register_module("pool1", torch::nn::MaxPool2d(2));

        conv2 = register_module("conv2", torch::nn::Conv2d(32, 64, 3));
        bn2   = register_module("bn2", torch::nn::BatchNorm2d(64));
        pool2 = register_module("pool2", torch::nn::MaxPool2d(2));

        conv3 = register_module("conv3", torch::nn::Conv2d(64, 128, 3));
        bn3   = register_module("bn3", torch::nn::BatchNorm2d(128));
        pool3 = register_module("pool3", torch::nn::MaxPool2d(2));

        flatten = register_module("flatten", torch::nn::Flatten());

        // ----- Classifier -----
        fc1 = register_module("fc1", torch::nn::Linear(128 * 30 * 30, 256));
        drop1 = register_module("drop1", torch::nn::Dropout(0.5));

        fc2 = register_module("fc2", torch::nn::Linear(256, 64));
        drop2 = register_module("drop2", torch::nn::Dropout(0.3));

        fc3 = register_module("fc3", torch::nn::Linear(64, 1));
    }

    torch::Tensor forward(torch::Tensor x)
    {
        // FEATURE EXTRACTOR
        x = torch::relu(bn1(conv1(x)));
        x = pool1(x);

        x = torch::relu(bn2(conv2(x)));
        x = pool2(x);

        x = torch::relu(bn3(conv3(x)));
        x = pool3(x);

        x = flatten(x);

        // CLASSIFIER
        x = torch::relu(fc1(x));
        x = drop1(x);

        x = torch::relu(fc2(x));
        x = drop2(x);

        x = torch::sigmoid(fc3(x));

        return x;
    }

private:
    torch::nn::Conv2d conv1{nullptr}, conv2{nullptr}, conv3{nullptr};
    torch::nn::BatchNorm2d bn1{nullptr}, bn2{nullptr}, bn3{nullptr};
    torch::nn::MaxPool2d pool1{nullptr}, pool2{nullptr}, pool3{nullptr};
    torch::nn::Flatten flatten{nullptr};

    torch::nn::Linear fc1{nullptr}, fc2{nullptr}, fc3{nullptr};
    torch::nn::Dropout drop1{nullptr}, drop2{nullptr};
};

TORCH_MODULE(CNN);

void train_model(CNN &model, torch::optim::Adam &optimizer,
                 std::vector<fs::path> &trainPaths, std::vector<uint8_t> &trainLabels,
                 int start_epoch, int num_epochs)
{
    auto cfg = get_loader_config();

    auto ds = ImageDataset(trainPaths, trainLabels).map(torch::data::transforms::Stack<>());
    auto loader = torch::data::make_data_loader(std::move(ds),
                                                torch::data::DataLoaderOptions().batch_size(cfg.batch_size).workers(cfg.workers));

    torch::nn::BCELoss criterion;

    for (int epoch = start_epoch; epoch < num_epochs; epoch++)
    {
        model->train();
        float running_loss = 0;
        size_t idx = 0;

        for (auto &batch : *loader)
        {
            auto data = batch.data.to(device);
            auto targets = batch.target.to(device).to(torch::kFloat32).unsqueeze(1);

            optimizer.zero_grad();
            auto outputs = model->forward(data);
            auto loss = criterion(outputs, targets);
            loss.backward();
            optimizer.step();

            running_loss += loss.item<float>();
            idx++;

            if (idx % 10 == 0)
                std::cout << "Epoch " << epoch + 1 << " step " << idx << " Loss " << loss.item<float>() << "\n";
        }

        std::cout << "Epoch " << epoch + 1 << " avg loss: " << running_loss / idx << "\n";
    }
}

float test_model(CNN &model, std::vector<fs::path> &testPaths, std::vector<uint8_t> &testLabels)
{
    auto cfg = get_loader_config();

    model->eval();
    torch::NoGradGuard no_grad;

    auto ds = ImageDataset(testPaths, testLabels).map(torch::data::transforms::Stack<>());
    auto loader = torch::data::make_data_loader(std::move(ds),
                                                torch::data::DataLoaderOptions().batch_size(cfg.batch_size).workers(cfg.workers));

    int correct = 0, total = 0;

    for (auto &batch : *loader)
    {
        auto data = batch.data.to(device);
        auto targets = batch.target.to(device).to(torch::kFloat32).unsqueeze(1);

        auto outputs = model->forward(data);
        auto preds = (outputs > 0.5).to(torch::kUInt8);

        correct += preds.eq(targets.to(torch::kUInt8)).sum().item<int>();
        total += targets.size(0);
    }

    float acc = 100.0f * correct / total;
    std::cout << "Test Accuracy: " << acc << "%\n";
    return acc;
}

int main()
{
    auto train_dir = fs::current_path() / "datasets/dogs-vs-cats/train";
    auto test_dir = fs::current_path() / "datasets/dogs-vs-cats/test";

    std::vector<fs::path> trainPaths, testPaths;
    std::vector<uint8_t> trainLabels, testLabels;
    std::map<std::string, uint8_t> classToLabel;

    load_dataset_folder(train_dir, trainPaths, trainLabels, classToLabel);
    load_dataset_folder(test_dir, testPaths, testLabels, classToLabel);

    if (torch::cuda::is_available())
        device = torch::kCUDA;

    CNN model;
    model->to(device);

    auto adam_options = torch::optim::AdamOptions(1e-4);
    torch::optim::Adam optimizer(model->parameters(), adam_options);

    fs::path checkpoint_path = "checkpoint.pt";

    int start_epoch = 0;
    int total_epochs = 5;

    if (fs::exists(checkpoint_path))
    {
        try
        {
            std::cout << "Loading checkpoint...\n";

            torch::serialize::InputArchive archive;
            archive.load_from(checkpoint_path.string());

            torch::serialize::InputArchive model_archive;
            archive.read("model", model_archive);
            model->load(model_archive);

            torch::serialize::InputArchive opt_archive;
            archive.read("optimizer", opt_archive);
            optimizer.load(opt_archive);

            torch::Tensor epoch_tensor;
            archive.read("epoch", epoch_tensor);

            start_epoch = epoch_tensor.item<int>();

            std::cout << "Checkpoint loaded. Starting from epoch " << start_epoch << "\n";

            test_model(model, testPaths, testLabels);
            return 0;
        }
        catch (const std::exception &e)
        {
            std::cout << "Failed loading checkpoint: " << e.what() << "\n";
        }
    }

    train_model(model, optimizer, trainPaths, trainLabels, start_epoch, total_epochs);

    try
    {
        torch::serialize::OutputArchive archive;
        // Save model
        {
            torch::serialize::OutputArchive model_archive;
            model->save(model_archive);
            archive.write("model", model_archive);
        }
        {
            torch::serialize::OutputArchive opt_archive;
            optimizer.save(opt_archive);
            archive.write("optimizer", opt_archive);
        }
        archive.write("epoch", torch::tensor(total_epochs));
        archive.save_to(checkpoint_path.string());

        std::cout << "Checkpoint saved to: " << checkpoint_path << "\n";
    }
    catch (const std::exception &e)
    {
        std::cout << "Failed saving checkpoint: " << e.what() << "\n";
    }

    test_model(model, testPaths, testLabels);
    return 0;
}


Overwriting Cats-Vs-Dogs-Classification.cpp


In [10]:
%%writefile CMakeLists.txt
cmake_minimum_required(VERSION 3.10 FATAL_ERROR)
project(CatsVsDogsClassification)

set(CMAKE_PREFIX_PATH "${CMAKE_SOURCE_DIR}/libtorch")
find_package(Torch REQUIRED)
find_package(OpenCV REQUIRED)

add_executable(CatsVsDogsClassification Cats-Vs-Dogs-Classification.cpp)
target_link_libraries(CatsVsDogsClassification "${TORCH_LIBRARIES}" ${OpenCV_LIBS})

set_property(TARGET CatsVsDogsClassification PROPERTY CXX_STANDARD 17)

Writing CMakeLists.txt


In [23]:
!mkdir -p /content/build
!cmake -DCMAKE_PREFIX_PATH=../libtorch -S /content/ -B /content/build/
!cmake --build /content/build/

-- PyTorch: CUDA detected: 12.5
-- PyTorch: CUDA nvcc is: /usr/local/cuda/bin/nvcc
-- PyTorch: CUDA toolkit directory: /usr/local/cuda
-- PyTorch: Header version is: 12.5
CMake Warning at libtorch/share/cmake/Caffe2/public/cuda.cmake:140 (message):
  Failed to compute shorthash for libnvrtc.so
Call Stack (most recent call first):
  libtorch/share/cmake/Caffe2/Caffe2Config.cmake:86 (include)
  libtorch/share/cmake/Torch/TorchConfig.cmake:68 (find_package)
  CMakeLists.txt:5 (find_package)


-- USE_CUDNN is set to 0. Compiling without cuDNN support
-- USE_CUSPARSELT is set to 0. Compiling without cuSPARSELt support
-- USE_CUDSS is set to 0. Compiling without cuDSS support
-- USE_CUFILE is set to 0. Compiling without cuFile support
-- Autodetected CUDA architecture(s):  7.5
-- Added CUDA NVCC flags for: -gencode;arch=compute_75,code=sm_75
-- Configuring done (1.7s)
-- Generating done (0.0s)
-- Build files have been written to: /content/build
[ 50%] Building CXX object CMakeFiles/CatsVsDog

In [21]:
!/content/build/CatsVsDogsClassification

Epoch 1 step 10 Loss 1.34401
Epoch 1 step 20 Loss 0.62356
Epoch 1 step 30 Loss 0.813285
Epoch 1 step 40 Loss 0.781002
Epoch 1 step 50 Loss 0.613833
Epoch 1 step 60 Loss 0.699204
Epoch 1 step 70 Loss 1.38337
Epoch 1 step 80 Loss 0.846219
Epoch 1 step 90 Loss 0.710473
Epoch 1 step 100 Loss 0.637344
Epoch 1 step 110 Loss 0.534698
Epoch 1 step 120 Loss 0.700761
Epoch 1 step 130 Loss 0.622504
Epoch 1 step 140 Loss 0.653945
Epoch 1 step 150 Loss 1.01264
Epoch 1 step 160 Loss 0.571148
Epoch 1 step 170 Loss 0.685942
Epoch 1 step 180 Loss 0.675222
Epoch 1 step 190 Loss 0.651658
Epoch 1 step 200 Loss 0.739668
Epoch 1 step 210 Loss 0.618488
Epoch 1 step 220 Loss 0.642688
Epoch 1 step 230 Loss 0.68739
Epoch 1 step 240 Loss 0.673578
Epoch 1 step 250 Loss 0.531061
Epoch 1 step 260 Loss 0.561661
Epoch 1 step 270 Loss 0.565138
Epoch 1 step 280 Loss 0.795166
Epoch 1 step 290 Loss 0.525172
Epoch 1 step 300 Loss 0.612956
Epoch 1 step 310 Loss 0.526463
Epoch 1 step 320 Loss 0.56901
Epoch 1 step 330 Loss 0